In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def get_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data['Close']

def calculate_returns(data):
    return data.pct_change().dropna()

def calculate_beta(stock_returns, market_returns):
    # 두 시리즈의 길이를 맞춤
    aligned_returns = pd.concat([stock_returns, market_returns], axis=1).dropna()
    if len(aligned_returns) < 2:
        return np.nan
    covariance = np.cov(aligned_returns.iloc[:, 0], aligned_returns.iloc[:, 1])[0][1]
    market_variance = np.var(aligned_returns.iloc[:, 1])
    if market_variance == 0:
        return np.nan
    return covariance / market_variance

def calculate_alpha(stock_returns, market_returns, beta):
    if np.isnan(beta):
        return np.nan
    aligned_returns = pd.concat([stock_returns, market_returns], axis=1).dropna()
    if len(aligned_returns) < 2:
        return np.nan
    return aligned_returns.iloc[:, 0].mean() - beta * aligned_returns.iloc[:, 1].mean()

def calculate_relative_strength(stock_data, kospi_data):
    aligned_data = pd.concat([stock_data, kospi_data], axis=1).dropna()
    if len(aligned_data) < 2:
        return np.nan
    return aligned_data.iloc[:, 0] / aligned_data.iloc[:, 1]

# 날짜 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# KOSPI 데이터 가져오기
kospi = get_stock_data("^KS11", start_date, end_date)

# 분석할 주식 리스트 (예시)
stocks = ["005930.KS", "000660.KS", "035720.KS", "329180.KS"]  # 삼성전자, SK하이닉스, 카카오

results = []

for stock in stocks:
    stock_data = get_stock_data(stock, start_date, end_date)
    
    # 수익률 계산
    stock_returns = calculate_returns(stock_data)
    kospi_returns = calculate_returns(kospi)
    
    # 베타 계산
    beta = calculate_beta(stock_returns, kospi_returns)
    
    # 알파 계산
    alpha = calculate_alpha(stock_returns, kospi_returns, beta)
    
    # 상대 강도 계산
    relative_strength = calculate_relative_strength(stock_data, kospi)
    
    # 총 수익률 계산
    total_return = (stock_data.iloc[-1] / stock_data.iloc[0] - 1) * 100 if len(stock_data) > 1 else np.nan
    
    # 결과 저장
    results.append({
        "Stock": stock,
        "Beta": beta,
        "Alpha": alpha,
        "Total Return": total_return,
        "Relative Strength": relative_strength.iloc[-1] if isinstance(relative_strength, pd.Series) else np.nan
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# KOSPI보다 강한 종목 필터링 (예: 알파가 양수이고 총 수익률이 KOSPI보다 높은 종목)
kospi_return = (kospi.iloc[-1] / kospi.iloc[0] - 1) * 100 if len(kospi) > 1 else np.nan
strong_stocks = results_df[(results_df['Alpha'] > 0) & (results_df['Total Return'] > kospi_return)].dropna()

print("KOSPI 대비 강세 종목:")
print(strong_stocks)

KOSPI 대비 강세 종목:
       Stock      Beta     Alpha  Total Return  Relative Strength
0  005930.KS  1.057172  0.000535     19.813566          29.278032
1  000660.KS  1.482639  0.001996     67.855376          72.503325
3  329180.KS  1.030016  0.002312     73.234811          78.897378
